<a href="https://colab.research.google.com/github/krumeto/fun_with_pandas/blob/master/Basketball_leaderboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --upgrade -q gspread
!pip install fuzzywuzzy[speedup]

     |████████████████████████████████| 51kB 3.3MB/s 
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144665 sha256=75e76d18688a8e3abbfdef66459a014ba332f14f66eb0106a2487a2cb6d60128
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-levenshtein


In [0]:
import pandas as pd
import re

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

In [0]:
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import numpy as np
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at drive


# NBA Fantasy Draft preparation - a Pandas exercise

A friend of mine is taking part in a NBA private fantasy draft and asked me for advice. I thought this would be a nice Pandas exercise - the data is online, requires some basic cleaning and getting it in a shape that is useful.

The goal is to get a ranking of players per basketball position, so that one can make decisions which player to choose during the draft. I am making the naive assumption that NBA players tend to perform similarly in back-to-back seasons and will only adjust slightly for above-32 seasons (5% penalty) and under-23 seasons (5% bonus). 

The scoring system used by the private league is:
    
    PTS + Rebounds + Assists*1.5 + Blocks*2 + Steals*2 + Double-Double*2 + Triple-Double*6 - TOV 
    = Total Points per Game

The final result should be a ranking per player and separate rankings per position.

#Getting data from Basketball reference

In [0]:
results_df = pd.read_html("https://www.basketball-reference.com/leagues/NBA_2019_per_game.html")[0]

In [0]:
#Taking a look at the data
results_df.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,.357,1.3,4.1,.323,0.5,1.0,.500,.487,0.4,0.4,.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,2,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,.222,0.2,1.5,.133,0.2,0.3,.667,.278,0.7,1.0,.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,3,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,.345,0.7,2.2,.338,0.4,1.1,.361,.459,0.2,0.3,.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,4,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,.595,0.0,0.0,.000,6.0,10.1,.596,.595,1.8,3.7,.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,5,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,.576,0.0,0.2,.200,3.4,5.7,.588,.579,2.0,2.8,.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9


In [0]:
#Drop the Rk column
results_df = results_df.drop(["Rk"], axis=1)

In [0]:
#Converting numerical columns to float

num_list = list(results_df.columns)
num_list = [num_col for num_col in num_list if num_col not in ('Rk','Player','Pos', 'Tm')]

for col in num_list:
  results_df[col] = pd.to_numeric(results_df[col], errors='coerce')
  
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 734 entries, 0 to 733
Data columns (total 29 columns):
Player    734 non-null object
Pos       734 non-null object
Age       708 non-null float64
Tm        734 non-null object
G         708 non-null float64
GS        708 non-null float64
MP        708 non-null float64
FG        708 non-null float64
FGA       708 non-null float64
FG%       702 non-null float64
3P        708 non-null float64
3PA       708 non-null float64
3P%       661 non-null float64
2P        708 non-null float64
2PA       708 non-null float64
2P%       693 non-null float64
eFG%      702 non-null float64
FT        708 non-null float64
FTA       708 non-null float64
FT%       665 non-null float64
ORB       708 non-null float64
DRB       708 non-null float64
TRB       708 non-null float64
AST       708 non-null float64
STL       708 non-null float64
BLK       708 non-null float64
TOV       708 non-null float64
PF        708 non-null float64
PTS       708 non-null float64

In [0]:
#Basketball Reference adds a line per team when a player is traded and a TOT line for the season performance. 
#Removing duplicate lines due to players being traded
traded_players = results_df.Player.value_counts() > 1
list_of_traded_players = list(traded_players[traded_players == True].index)# Getting a list of players appearing multiple times
print(list_of_traded_players[:10])

['Player', 'Greg Monroe', 'Andrew Harrison', 'Isaiah Canaan', 'Alec Burks', 'Wesley Matthews', 'Jason Smith', 'Mike Scott', 'Wayne Selden', 'Brandon Knight']


In [0]:
for player in list_of_traded_players:
    indexes = results_df[results_df.Player == player][results_df.Tm != "TOT"].index
    results_df.drop(indexes, inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [0]:
#Check if there are still players with multiple entries
results_df.Player.value_counts().head()

Cory Joseph         1
Carmelo Anthony     1
Antonio Blakeney    1
Tyler Davis         1
Kent Bazemore       1
Name: Player, dtype: int64

## Double-Doubles and Triple-Doubles

The scoring for this particular fantasy league gives a significant bonus for Double-doubles  and Triple-doubles. Estimating them properly is very important. 

In [0]:
#Get data online on DDs and TDs for the previous season
doubles = pd.read_html("https://www.landofbasketball.com/year_by_year_stats/2018_2019_double_doubles_rs.htm", header=1)
#Header=1 indicates the row on which the header is located
#First table is all we need
doubles = doubles[0]
doubles.head()

,Rk,Player (Team),Double-Doubles,Gms,Triple-Doubles,PtsReb,PtsAst,RebAst,Other
0,1,Andre Drummond (Pistons),69,79,-,69,-,-,-
1,2,Rudy Gobert (Jazz),66,81,-,66,-,-,-
2,3,Nikola Vucevic (Magic),60,80,1,59,-,-,-
3,4,Joel Embiid (76ers),58,64,2,56,-,-,-
4,5,Russell Westbrook (Thunder),57,73,34,15,8,-,-


In [0]:
#Since only the name, DDs and TDs are needed, I will select only those.
doubles = doubles.iloc[:,1:5]
doubles.head()

,Player (Team),Double-Doubles,Gms,Triple-Doubles
0,Andre Drummond (Pistons),69,79,-
1,Rudy Gobert (Jazz),66,81,-
2,Nikola Vucevic (Magic),60,80,1
3,Joel Embiid (76ers),58,64,2
4,Russell Westbrook (Thunder),57,73,34


In [0]:
#There are two problems left - the names of the players include their team, which would preclude us from merging tables and the double double column includes the triple doubles.

doubles['Player'] = doubles['Player  (Team)'].str.split(" \(", expand=True)[0]
doubles = doubles.iloc[:,1:]
doubles.head()

,Double-Doubles,Gms,Triple-Doubles,Player
0,69,79,-,Andre Drummond
1,66,81,-,Rudy Gobert
2,60,80,1,Nikola Vucevic
3,58,64,2,Joel Embiid
4,57,73,34,Russell Westbrook


In [0]:
#Before tackling the issue with Double-doubles, I need to convert the columns to numeric. Before that, some errors need to be cleaned
doubles[~doubles['Double-Doubles'].str.isnumeric()]

,Double-Doubles,Gms,Triple-Doubles,Player
20,Double-Doubles,Gms,Triple-Doubles,Player
43,Double-Doubles,Gms,Triple-Doubles,Player
65,Double-Doubles,Gms,Triple-Doubles,Player
89,Double-Doubles,Gms,Triple-Doubles,Player
114,Double-Doubles,Gms,Triple-Doubles,Player
163,Double-Doubles,Gms,Triple-Doubles,Player
194,Double-Doubles,Gms,Triple-Doubles,Player


In [0]:
#The above rows clearly can be deleted
doubles = doubles[doubles['Double-Doubles'].str.isnumeric()]

In [0]:
doubles.tail()

,Double-Doubles,Gms,Triple-Doubles,Player
263,1,66,-,Daniel Theis
264,1,51,-,Ekpe Udoh
265,1,64,-,Fred VanVleet
266,1,46,-,Kenrich Williams
267,2358,NaN,NaN,Total:


In [0]:
#Last line is a total - I will remove it as well
doubles = doubles[:-1]

In [0]:
doubles[~doubles['Triple-Doubles'].str.isnumeric()]['Triple-Doubles'].unique()

array(['-'], dtype=object)

In [0]:
#It seems there is only one place-holder for zero '-'. I will just replace it with zero
doubles['Triple-Doubles'] = doubles['Triple-Doubles'].str.replace('-', "0")

In [0]:
doubles = doubles.apply(pd.to_numeric, errors='ignore') #'Ignore' so that we do not lose the players' names
doubles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260 entries, 0 to 266
Data columns (total 4 columns):
Double-Doubles    260 non-null int64
Gms               260 non-null int64
Triple-Doubles    260 non-null int64
Player            260 non-null object
dtypes: int64(3), object(1)
memory usage: 20.2+ KB


In [0]:
doubles['Double-Doubles'] = doubles['Double-Doubles'] - doubles['Triple-Doubles']
doubles.head(10)

,Double-Doubles,Gms,Triple-Doubles,Player
0,69,79,0,Andre Drummond
1,66,81,0,Rudy Gobert
2,59,80,1,Nikola Vucevic
3,56,64,2,Joel Embiid
4,23,73,34,Russell Westbrook
5,44,80,12,Nikola Jokic
6,49,72,5,Giannis Antetokounmpo
7,54,77,0,Karl-Anthony Towns
8,45,67,0,Clint Capela
9,42,69,0,DeAndre Jordan


In [0]:
#Next, I will calculate the portion of games each player played and got a DD or a TD
doubles['Double-Doubles-Portion'] = doubles['Double-Doubles']/doubles['Gms']
doubles['Triple-Doubles-Portion'] = doubles['Triple-Doubles']/doubles['Gms']
doubles.head()

,Double-Doubles,Gms,Triple-Doubles,Player,Double-Doubles-Portion,Triple-Doubles-Portion
0,69,79,0,Andre Drummond,0.873418,0.000000
1,66,81,0,Rudy Gobert,0.814815,0.000000
2,59,80,1,Nikola Vucevic,0.737500,0.012500
3,56,64,2,Joel Embiid,0.875000,0.031250
4,23,73,34,Russell Westbrook,0.315068,0.465753


There is an issue with the names strings in the Player column - there is a whitespace at the end:

In [0]:
doubles.Player[1]

'Rudy Gobert '

In [0]:
doubles.Player = doubles.Player.apply(lambda x: re.sub("[ \t]+$", "", x))

In [0]:
doubles.Player[1]

'Rudy Gobert'

In [0]:
doubles.Player = doubles.Player.str.lower()
results_df.Player = results_df.Player.str.lower()

In [0]:
print(results_df.shape)
results_df_with_double_doubles = results_df.merge(doubles,how='left', on="Player")
print(results_df_with_double_doubles.shape)
results_df_with_double_doubles.head()

(530, 29)
(530, 34)


,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Double-Doubles,Gms,Triple-Doubles,Double-Doubles-Portion,Triple-Doubles-Portion
0,álex abrines,SG,25.0,OKC,31.0,2.0,19.0,1.8,5.1,0.357,1.3,4.1,0.323,0.5,1.0,0.500,0.487,0.4,0.4,0.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3,NaN,NaN,NaN,NaN,NaN
1,quincy acy,PF,28.0,PHO,10.0,0.0,12.3,0.4,1.8,0.222,0.2,1.5,0.133,0.2,0.3,0.667,0.278,0.7,1.0,0.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7,NaN,NaN,NaN,NaN,NaN
2,jaylen adams,PG,22.0,ATL,34.0,1.0,12.6,1.1,3.2,0.345,0.7,2.2,0.338,0.4,1.1,0.361,0.459,0.2,0.3,0.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2,NaN,NaN,NaN,NaN,NaN
3,steven adams,C,25.0,OKC,80.0,80.0,33.4,6.0,10.1,0.595,0.0,0.0,0.000,6.0,10.1,0.596,0.595,1.8,3.7,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9,29.0,80.0,0.0,0.362500,0.0
4,bam adebayo,C,21.0,MIA,82.0,28.0,23.3,3.4,5.9,0.576,0.0,0.2,0.200,3.4,5.7,0.588,0.579,2.0,2.8,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9,11.0,82.0,0.0,0.134146,0.0


In [0]:
len(doubles['Double-Doubles'])

260

In [0]:
players_1 = results_df_with_double_doubles[results_df_with_double_doubles['Double-Doubles'].notna()].Player.unique()
player_2 = doubles.Player.unique()

In [0]:
set1 = set(players_1)
set2 = set(player_2)

missing = list(sorted(set1 - set2))
added = list(sorted(set2 - set1))

print('missing:', missing)
print('added:', added)

missing: []
added: ['ante zizic', 'boban marjanovic', 'bogdan bogdanovic', 'c.j. mccollum', 'dario saric', 'dennis schroder', 'dennis smith jr.', 'ersan ilyasova', 'goran dragic', 'jakob poeltl', 'jaren jackson jr.', 'jonas valanciunas', 'jose juan barea', 'jusuf nurkic', 'kelly oubre jr.', 'larry nance jr.', 'luka doncic', 'marvin bagley iii', 'mohamed bamba', 'nikola jokic', 'nikola mirotic', 'nikola vucevic', 'skal labissiere', 'tim hardaway jr.', 'timothe luwawu-cabarrot', 'tomas satoransky', 'wendell carter jr.', 'willy hernangomez']


In [0]:
results_df[results_df.Player.str.contains("timo")]

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
429,timothé luwawu-cabarrot,SF,23.0,TOT,50.0,7.0,13.4,1.6,4.4,0.376,0.7,2.3,0.31,0.9,2.0,0.451,0.459,0.6,0.8,0.756,0.2,1.7,1.9,0.5,0.4,0.2,0.4,1.5,4.6


In [0]:
def get_closest_name(players_name):
  return process.extractOne(players_name, results_df.Player.unique())[0]

In [0]:
replacement_dict = {}
for missing_player in added:
  replacement_dict[get_closest_name(missing_player)] = missing_player
replacement_dict

{'ante žižić': 'ante zizic',
 'boban marjanović': 'boban marjanovic',
 'bogdan bogdanović': 'bogdan bogdanovic',
 'cj mccollum': 'c.j. mccollum',
 'dario šarić': 'dario saric',
 'dennis schröder': 'dennis schroder',
 'dennis smith': 'dennis smith jr.',
 'ersan i̇lyasova': 'ersan ilyasova',
 'goran dragić': 'goran dragic',
 'j.j. barea': 'jose juan barea',
 'jakob pöltl': 'jakob poeltl',
 'jaren jackson': 'jaren jackson jr.',
 'jonas valančiūnas': 'jonas valanciunas',
 'jusuf nurkić': 'jusuf nurkic',
 'kelly oubre': 'kelly oubre jr.',
 'larry nance': 'larry nance jr.',
 'luka dončić': 'luka doncic',
 'marvin bagley': 'marvin bagley iii',
 'mo bamba': 'mohamed bamba',
 'nikola jokić': 'nikola jokic',
 'nikola mirotić': 'nikola mirotic',
 'nikola vučević': 'nikola vucevic',
 'skal labissière': 'skal labissiere',
 'tim hardaway': 'tim hardaway jr.',
 'timothé luwawu-cabarrot': 'timothe luwawu-cabarrot',
 'tomáš satoranský': 'tomas satoransky',
 'wendell carter': 'wendell carter jr.',
 'wil

In [0]:
results_df.Player = results_df.Player.replace(replacement_dict)

In [0]:
results_df[results_df.Player.str.contains("zizic")]

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
730,ante zizic,C,22.0,CLE,59.0,25.0,18.3,3.1,5.6,0.553,0.0,0.0,NaN,3.1,5.6,0.553,0.553,1.6,2.2,0.705,1.8,3.6,5.4,0.9,0.2,0.4,1.0,1.9,7.8


In [0]:
print(results_df.shape)
results_df_with_double_doubles = results_df.merge(doubles,how='left', on="Player")
print(results_df_with_double_doubles.shape)
results_df_with_double_doubles.head()
len(results_df_with_double_doubles[results_df_with_double_doubles['Double-Doubles'].notna()].Player.unique())

(530, 29)
(530, 34)


260

In [0]:
results_df_with_double_doubles['Below_23'] = np.where(results_df_with_double_doubles.Age < 23 , 1.05, 1)
results_df_with_double_doubles['Above_31'] = np.where(results_df_with_double_doubles.Age > 31 , 0.95, 1)
results_df_with_double_doubles.head(10)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Double-Doubles,Gms,Triple-Doubles,Double-Doubles-Portion,Triple-Doubles-Portion,Below_23,Above_31
0,álex abrines,SG,25.0,OKC,31.0,2.0,19.0,1.8,5.1,0.357,1.3,4.1,0.323,0.5,1.0,0.500,0.487,0.4,0.4,0.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3,NaN,NaN,NaN,NaN,NaN,1.00,1.00
1,quincy acy,PF,28.0,PHO,10.0,0.0,12.3,0.4,1.8,0.222,0.2,1.5,0.133,0.2,0.3,0.667,0.278,0.7,1.0,0.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7,NaN,NaN,NaN,NaN,NaN,1.00,1.00
2,jaylen adams,PG,22.0,ATL,34.0,1.0,12.6,1.1,3.2,0.345,0.7,2.2,0.338,0.4,1.1,0.361,0.459,0.2,0.3,0.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2,NaN,NaN,NaN,NaN,NaN,1.05,1.00
3,steven adams,C,25.0,OKC,80.0,80.0,33.4,6.0,10.1,0.595,0.0,0.0,0.000,6.0,10.1,0.596,0.595,1.8,3.7,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9,29.0,80.0,0.0,0.362500,0.0,1.00,1.00
4,bam adebayo,C,21.0,MIA,82.0,28.0,23.3,3.4,5.9,0.576,0.0,0.2,0.200,3.4,5.7,0.588,0.579,2.0,2.8,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9,11.0,82.0,0.0,0.134146,0.0,1.05,1.00
5,deng adel,SF,21.0,CLE,19.0,3.0,10.2,0.6,1.9,0.306,0.3,1.2,0.261,0.3,0.7,0.385,0.389,0.2,0.2,1.000,0.2,0.8,1.0,0.3,0.1,0.2,0.3,0.7,1.7,NaN,NaN,NaN,NaN,NaN,1.05,1.00
6,devaughn akoon-purcell,SG,25.0,DEN,7.0,0.0,3.1,0.4,1.4,0.300,0.0,0.6,0.000,0.4,0.9,0.500,0.300,0.1,0.3,0.500,0.1,0.4,0.6,0.9,0.3,0.0,0.3,0.6,1.0,NaN,NaN,NaN,NaN,NaN,1.00,1.00
7,lamarcus aldridge,C,33.0,SAS,81.0,81.0,33.2,8.4,16.3,0.519,0.1,0.5,0.238,8.3,15.8,0.528,0.522,4.3,5.1,0.847,3.1,6.1,9.2,2.4,0.5,1.3,1.8,2.2,21.3,32.0,81.0,0.0,0.395062,0.0,1.00,0.95
8,rawle alkins,SG,21.0,CHI,10.0,1.0,12.0,1.3,3.9,0.333,0.3,1.2,0.250,1.0,2.7,0.370,0.372,0.8,1.2,0.667,1.1,1.5,2.6,1.3,0.1,0.0,0.8,0.7,3.7,NaN,NaN,NaN,NaN,NaN,1.05,1.00
9,grayson allen,SG,23.0,UTA,38.0,2.0,10.9,1.8,4.7,0.376,0.8,2.6,0.323,0.9,2.1,0.443,0.466,1.2,1.6,0.750,0.1,0.5,0.6,0.7,0.2,0.2,0.9,1.2,5.6,NaN,NaN,NaN,NaN,NaN,1.00,1.00


In [0]:
results_df_with_double_doubles = results_df_with_double_doubles.fillna(0)

In [0]:
#Creating the fantasy score
results_df_with_double_doubles['fantasy_score'] = results_df_with_double_doubles.PTS + results_df_with_double_doubles.TRB + results_df_with_double_doubles.AST*1.5 + results_df_with_double_doubles.BLK*2 + results_df_with_double_doubles.STL*2 + results_df_with_double_doubles["Double-Doubles-Portion"]*2 + results_df_with_double_doubles["Triple-Doubles-Portion"]*6 -  results_df_with_double_doubles.TOV

In [0]:
results_df_with_double_doubles.tail()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Double-Doubles,Gms,Triple-Doubles,Double-Doubles-Portion,Triple-Doubles-Portion,Below_23,Above_31,fantasy_score
525,robert williams,C,21.0,BOS,32.0,2.0,8.8,1.1,1.6,0.706,0.0,0.0,0.000,1.1,1.6,0.706,0.706,0.3,0.5,0.600,0.8,1.7,2.5,0.2,0.3,1.3,0.3,1.1,2.5,0.0,0.0,0.0,0.0,0.0,1.05,1.00,8.20
526,troy williams,SF,24.0,SAC,21.0,0.0,14.9,2.1,4.7,0.449,0.7,2.1,0.318,1.4,2.6,0.556,0.520,0.4,0.7,0.600,0.6,2.2,2.8,0.5,0.5,0.4,0.4,1.8,5.3,0.0,0.0,0.0,0.0,0.0,1.00,1.00,10.25
527,guerschon yabusele,PF,23.0,BOS,41.0,1.0,6.1,0.9,1.9,0.455,0.2,0.7,0.321,0.6,1.2,0.531,0.513,0.4,0.5,0.682,0.6,0.7,1.3,0.4,0.2,0.2,0.4,0.8,2.3,0.0,0.0,0.0,0.0,0.0,1.00,1.00,4.60
528,nick young,SG,33.0,DEN,4.0,0.0,9.3,0.8,2.3,0.333,0.8,2.0,0.375,0.0,0.3,0.000,0.500,0.0,0.0,0.000,0.0,0.3,0.3,0.5,0.0,0.3,0.5,1.0,2.3,0.0,0.0,0.0,0.0,0.0,1.00,0.95,3.45
529,tyler zeller,C,29.0,TOT,6.0,1.0,15.5,2.7,5.0,0.533,0.0,0.2,0.000,2.7,4.8,0.552,0.533,2.3,3.0,0.778,1.8,2.2,4.0,0.7,0.2,0.5,0.7,3.3,7.7,0.0,0.0,0.0,0.0,0.0,1.00,1.00,13.45


In [0]:
results_df_with_double_doubles['fantasy_score'] = (results_df_with_double_doubles['fantasy_score'])*results_df_with_double_doubles['Below_23']*results_df_with_double_doubles['Above_31']

#This is a preliminary list, sorted by column 'fantasy_score'

Challanges remaining:
1. Assess the importance of games played
2. Assess the importance of USG rate


In [0]:
#Sorting the DataFrame 
results_df_with_double_doubles = results_df_with_double_doubles.sort_values('fantasy_score', ascending=False)
results_df_with_double_doubles = results_df_with_double_doubles.reset_index(drop=True)
results_df_with_double_doubles[['Player', "Pos", 'fantasy_score']]

,Player,Pos,fantasy_score
0,james harden,PG,55.580769
1,russell westbrook,PG,53.774658
2,giannis antetokounmpo,PF,52.727778
3,anthony davis,C,51.071429
4,joel embiid,C,50.287500
5,lebron james,SF,47.780682
6,paul george,SF,45.525325
7,karl-anthony towns,C,45.202597
8,nikola jokic,C,44.950000
9,kevin durant,SF,42.462821


#Top per position

In [0]:
for position in results_df_with_double_doubles.Pos.unique():
  print("Top 10 players for " + position)
  print(results_df_with_double_doubles.groupby("Pos").get_group(position)[['Player', 'fantasy_score']].head(10))
  print('-------------------------------------------')

Top 10 players for PG
               Player  fantasy_score
0        james harden      55.580769
1   russell westbrook      53.774658
12        ben simmons      41.705601
13     damian lillard      41.375000
16       kyrie irving      41.236567
17      stephen curry      41.086957
21       kemba walker      39.693902
23          john wall      38.975000
26   d'angelo russell      37.322963
28         trae young      35.847130
-------------------------------------------
Top 10 players for PF
                   Player  fantasy_score
2   giannis antetokounmpo      52.727778
22          blake griffin      39.566667
32          julius randle      35.454110
33           john collins      35.016639
42        lauri markkanen      33.147692
43          tobias harris      32.865854
47             kevin love      31.390909
54          pascal siakam      30.200000
55           aaron gordon      30.137179
70             kyle kuzma      28.164286
-------------------------------------------
Top 10 pla

In [0]:
results_df_with_double_doubles.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Double-Doubles',
       'Gms', 'Triple-Doubles', 'Double-Doubles-Portion',
       'Triple-Doubles-Portion', 'Below_23', 'Above_31', 'fantasy_score'],
      dtype='object')

In [0]:
small_df = results_df_with_double_doubles[['Player', 'Pos', 'Age', 'G', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Double-Doubles',
        'Triple-Doubles','fantasy_score']]

In [0]:
small_df

,Player,Pos,Age,G,TRB,AST,STL,BLK,TOV,PF,PTS,Double-Doubles,Triple-Doubles,fantasy_score
0,james harden,PG,29.0,78.0,6.6,7.5,2.0,0.7,5.0,3.1,36.1,27.0,7.0,55.580769
1,russell westbrook,PG,30.0,73.0,11.1,10.7,1.9,0.5,4.5,3.4,22.9,23.0,34.0,53.774658
2,giannis antetokounmpo,PF,24.0,72.0,12.5,5.9,1.3,1.5,3.7,3.2,27.7,49.0,5.0,52.727778
3,anthony davis,C,25.0,56.0,12.0,3.9,1.6,2.4,2.0,2.4,25.9,37.0,0.0,51.071429
4,joel embiid,C,24.0,64.0,13.6,3.7,0.7,1.9,3.5,3.3,27.5,56.0,2.0,50.287500
5,lebron james,SF,34.0,55.0,8.5,8.3,1.3,0.6,3.6,1.7,27.4,24.0,8.0,47.780682
6,paul george,SF,28.0,77.0,8.2,4.1,2.2,0.4,2.7,2.8,28.0,23.0,1.0,45.525325
7,karl-anthony towns,C,23.0,77.0,12.4,3.4,0.9,1.6,3.1,3.8,24.4,54.0,0.0,45.202597
8,nikola jokic,C,23.0,80.0,10.8,7.3,1.4,0.7,3.1,2.9,20.1,44.0,12.0,44.950000
9,kevin durant,SF,30.0,78.0,6.4,5.9,0.7,1.1,2.9,2.0,26.0,14.0,2.0,42.462821


In [0]:
small_df.shape

(530, 14)

In [0]:
small_df.tail()

,Player,Pos,Age,G,TRB,AST,STL,BLK,TOV,PF,PTS,Double-Doubles,Triple-Doubles,fantasy_score
525,okaro white,PF,26.0,3.0,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7
526,john holland,SF,30.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
527,kobi simmons,PG,21.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
528,tyler ulis,PG,23.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
529,donte grantham,SF,23.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Uploading to Google Sheets

In [0]:
small_df.to_excel('Basketball fantasy draft board.xlsx', index=False)
!cp 'Basketball fantasy draft board.xlsx' drive/My\ Drive/